In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore", category=sm.tools.sm_exceptions.ConvergenceWarning)

# Cargar los datos
data = pd.read_csv("Session6Data.csv", sep=";", decimal=",")
y = data.iloc[:, 3]

long = len(y)
n_estimation = 400 # training sample
real = y[(n_estimation):len(y)] # test sample

n_forecasting = long - n_estimation # number of horizons we can predict
horizontes = 2 # we choose 2 in this example, but can be any value you want

predicc = np.zeros((horizontes, n_forecasting))
MSFE = np.zeros((horizontes, 1))
MAPE = np.zeros((horizontes, 1))

for Periods_ahead in range(horizontes):
    for i in range(0, n_forecasting):
        aux_y = y[0:(n_estimation - Periods_ahead + i)] # Recursive
        # aux_y = y[i:(n_estimation - Periods_ahead + i)] # Rolling
        model = ARIMA(aux_y, order=(1, 1, 0)) # original scale
        # model = ARIMA(np.log(aux_y), order=(1, 1, 0)) # with the log transformation
        model_fit = model.fit()
        y_pred = model_fit.forecast(steps=Periods_ahead + 1)
        predicc[Periods_ahead][i] = y_pred.iloc[-1] # original scale
        # predicc[Periods_ahead][i] = np.exp(y_pred.iloc[-1])  # with the log transformation
    # Calcular errores
    error = np.array(real) - predicc[Periods_ahead]
    MSFE[Periods_ahead] = np.mean(error**2)
    MAPE[Periods_ahead] = np.mean(np.abs(error / np.array(real))) * 100

# Crear DataFrame de predicciones
df_pred = pd.DataFrame({"V1": predicc[0], "V2": predicc[1]})

print("MSFE", MSFE)
print("MAPE %", MAPE)


MSFE [[ 4.12040251]
 [12.556055  ]]
MAPE % [[2.62314869]
 [4.74441081]]
